# **import required libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os
from torchvision import models
from google.colab import drive
!pip install torchsummary
from torchsummary import summary
from PIL import Image
import shutil
import random
from collections import Counter


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Define Dataset Paths**

In [ ]:
# ✅ Define dataset paths
dataset_path = "/content/drive/My Drive/plant_diseasea dataset"
train_dir = os.path.join(dataset_path, "New Plant Diseases Dataset(Augmented)/train")
valid_dir = os.path.join(dataset_path, "New Plant Diseases Dataset(Augmented)/valid")

# ✅ Check if dataset exists
if os.path.exists(train_dir) and os.path.exists(valid_dir):
    print("✅ Dataset Found!")
else:
    print("❌ Dataset not found. Check Google Drive path.")

✅ Dataset Found!


# **Check Dataset Classes & Distribution**

In [ ]:
train_classes = os.listdir(train_dir)
valid_classes = os.listdir(valid_dir)

print(f"✅ Training Classes: {train_classes}")
print(f"✅ Validation Classes: {valid_classes}")
print(f"✅ Number of Classes: {len(train_classes)}")

# ✅ Check for unexpected classes
expected_classes = [
    "Apple___Apple_scab", "Apple___Black_rot", "Apple___Cedar_apple_rust", "Apple___healthy",
    "Grape___Black_rot", "Grape___Esca_(Black_Measles)", "Grape___healthy", "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)",
    "Potato___Early_blight", "Potato___healthy", "Potato___Late_blight"
]

unexpected_classes = [c for c in train_classes if c not in expected_classes]

if unexpected_classes:
    print(f"⚠️ Unexpected Classes Found: {unexpected_classes}")
else:
    print("✅ Dataset is Clean and Contains Only the 11 Expected Classes.")


✅ Training Classes: ['Potato___healthy', 'Potato___Early_blight', 'Grape___healthy', 'Grape___Black_rot', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Potato___Late_blight', 'Apple___Cedar_apple_rust', 'Grape___Esca_(Black_Measles)', 'Apple___healthy', 'Apple___Black_rot', 'Apple___Apple_scab']
✅ Validation Classes: ['Apple___Black_rot', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Grape___Esca_(Black_Measles)', 'Grape___Black_rot', 'Potato___healthy', 'Potato___Late_blight', 'Potato___Early_blight', 'Apple___healthy', 'Apple___Cedar_apple_rust', 'Apple___Apple_scab']
✅ Number of Classes: 11
✅ Dataset is Clean and Contains Only the 11 Expected Classes.


#  Balance Dataset - Move Excess Validation Images to *Training*

In [ ]:
# ✅ Target validation size (~350 images)
target_valid_size = 350

for class_folder in os.listdir(valid_dir):
    valid_class_path = os.path.join(valid_dir, class_folder)
    train_class_path = os.path.join(train_dir, class_folder)

    if os.path.isdir(valid_class_path):
        images = os.listdir(valid_class_path)
        if len(images) > target_valid_size / len(os.listdir(valid_dir)):
            images_to_move = random.sample(images, len(images) - target_valid_size // len(os.listdir(valid_dir)))
            for img in images_to_move:
                shutil.move(os.path.join(valid_class_path, img), train_class_path)

print("✅ Dataset successfully rebalanced. Training and validation sizes adjusted!")


✅ Dataset successfully rebalanced. Training and validation sizes adjusted!


# **Count Training & Validation Samples**

In [ ]:
def count_images(folder_path):
    return sum([len(files) for _, _, files in os.walk(folder_path)])

train_count = count_images(train_dir)
valid_count = count_images(valid_dir)

print(f"✅ Training Samples: {train_count}")
print(f"✅ Validation Samples: {valid_count}")

if valid_count < 0.1 * train_count:
    print("⚠️ Warning: Validation set is too small. Consider re-splitting dataset!")


✅ Training Samples: 2165
✅ Validation Samples: 341


# **Define Transformations**

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Reduce from 224x224 to 128x128
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



# **Load Dataset Using PyTorch**

In [ ]:
# ✅ Load dataset
train_data = ImageFolder(train_dir, transform=transform)
valid_data = ImageFolder(valid_dir, transform=transform)

# ✅ Define batch size
batch_size = 64

# ✅ Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)

print(f"✅ Training Samples: {len(train_data)}")
print(f"✅ Validation Samples: {len(valid_data)}")
print(f"✅ Number of Classes: {len(train_data.classes)}")


✅ Training Samples: 2165
✅ Validation Samples: 341
✅ Number of Classes: 11


# **Define ResNet Model**

In [ ]:
# ✅ Load Pretrained Model
model = models.resnet50(pretrained=True)  # ResNet-50 (better than ResNet-18)

# ✅ Modify Final Layer to match the number of classes
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# ✅ Move Model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# ✅ Print Model Summary
summary(model, (3, 128, 128))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]           4,096
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 256, 32, 32]          16,384
      BatchNorm2d-12          [-1, 256, 32, 32]             512
           Conv2d-13          [-1, 256, 32, 32]          16,384
      BatchNorm2d-14          [-1, 256,

# **Define Loss Function & Optimizer**

In [ ]:
# ✅ Define Loss Function
criterion = nn.CrossEntropyLoss()

# ✅ Define Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Lower LR for stability


# **Train the Model**

In [ ]:
# ✅ Train for 10 epochs
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_accuracy = correct / total * 100
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

print("✅ Training Completed!")


Epoch [1/5], Loss: 0.6579, Accuracy: 99.45%
Epoch [2/5], Loss: 0.2344, Accuracy: 99.91%
Epoch [3/5], Loss: 0.1830, Accuracy: 99.91%
Epoch [4/5], Loss: 0.0791, Accuracy: 100.00%
Epoch [5/5], Loss: 0.0407, Accuracy: 100.00%
✅ Training Completed!


# **Evaluate Model on Validation Data**

In [ ]:
# ✅ Validate Model
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

validation_accuracy = correct / total * 100
print(f"✅ Validation Accuracy: {validation_accuracy:.2f}%")


✅ Validation Accuracy: 98.24%


# **Save the Model**

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/plant_disease_model_latest.pth")
print("✅ Model Saved Successfully!")


✅ Model Saved Successfully!


In [ ]:
import torch
# Load model from Colab
model_path = "/content/drive/My Drive/plant_disease_model_latest.pth"
try:
    model_colab = torch.load(model_path)
    print("✅ Model in Colab loaded successfully!")
except Exception as e:
    print(f"❌ Model in Colab is corrupt: {e}")


✅ Model in Colab loaded successfully!


<ipython-input-96-f19940ca2c24>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_colab = torch.load(model_path)


In [ ]:
!cp "/content/plant_disease_model_latest.pth" "/content/drive/My Drive/"
print("✅ Model copied to Google Drive successfully!")


✅ Model copied to Google Drive successfully!


In [ ]:
import torch
print(f"✅ PyTorch Version in Colab: {torch.__version__}")


✅ PyTorch Version in Colab: 2.5.1+cu124


# **Test on a New Image**

In [ ]:
# ✅ Load an image from validation dataset
image_path = "/content/drive/My Drive/plant_diseasea dataset/New Plant Diseases Dataset(Augmented)/valid/Grape___Black_rot/08a7300a-1e67-4441-9521-9168d47cd665___FAM_B.Rot 3020_flipLR.JPG"

if os.path.exists(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)

    # ✅ Predict Class
    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)

    print(f"✅ Predicted Disease: {train_data.classes[predicted_class.item()]}")
else:
    print(f"❌ Image not found at: {image_path}")


✅ Predicted Disease: Grape___Black_rot
